# Asadullah Aliyev _ Capstone project _ week 3 _ Part 2

## From first submission

Below codes were briefly discussed in this week's first submission - https://github.com/asadAI/asad_ai_w3_tasks/blob/master/Capstone_w3_1.ipynb . In order to run the second task succesfully, we should initially begin with them.


In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content, "html.parser")
soup = soup.find_all("table", {"class":"wikitable sortable"})[0].find_all("td")
df = pd.DataFrame(columns= ["Postcode", "Borough","Neighborhood"]) # empty dataframe

for i in range(len(soup)):  # len(soup) stands for number of postcode, borough and neighborhood combinations
     if i % 3==0:  
        df= df.append({'Postcode': soup[i].text, 
                        'Borough': soup[i+1].text, 
                        'Neighborhood': str(soup[i+2].text).replace("\n","")},ignore_index=True)
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood' ] =df.loc[df['Neighborhood'] == 'Not assigned','Borough' ]
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(inplace=True, drop=True)
# combine neighborhoods with same postal code area
for j in range(1,len(df)):
    if df['Postcode'].iloc[j]==df['Postcode'].iloc[j-1] and df['Borough'].iloc[j]==df['Borough'].iloc[j-1]:
        df['Neighborhood'].iloc[j]=df['Neighborhood'].iloc[j] + ", " + df['Neighborhood'].iloc[j-1]
        

# delete the duplicated row

df['Repeat']=""  # add an artificial column 'Repeat' for duplicated items
for j in range(0,len(df)-1):
    if df['Postcode'].iloc[j]==df['Postcode'].iloc[j+1] and df['Borough'].iloc[j]==df['Borough'].iloc[j+1]:
        df['Repeat'].iloc[j]=1   # type 1 against rows to be deleted

df.drop(df.loc[df['Repeat']==1].index, inplace=True) # delete rows containing 1 under 'Repeat' column
df.drop(['Repeat'], axis=1, inplace=True)   # delete the 'Repeat' column since now there is nothing to do with it
df.reset_index(inplace=True, drop=True)
print("rows, columns:", df.shape)
df.head()



rows, columns: (103, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


## Let's start to second submission

Since we are not able to get the geographical coordinates of the neighborhoods using the Geocoder package,  we will ise a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [29]:
df1=pd.read_csv("http://cocl.us/Geospatial_data")
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In order to inner join two dataframes, they should have at least one similar column. Therefore, let's change the name of column **"Postcal Code"** to **"Postcode"** in second dataframe df1.


In [30]:
df1 = df1.rename(columns={'Postal Code': 'Postcode'})
df1.head()


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


And now it is time to inner join dataframes df and df1 under df2.

In [34]:
df2=pd.merge(df, df1, on='Postcode')
print("rows, columns:", df2.shape)
df2.head(11)

rows, columns: (103, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
